In [1]:
import numpy as np
import math
import scipy.io.wavfile
from my_util import load_obj, unit_vector, angle_between
from scipy.linalg import eig, norm
import matplotlib.pyplot as plt
from pickle import dump, load

# Load mesh and construct matrices

In [21]:
mesh_file = "meshes/railing.obj"

In [22]:
# Material properties
# Glass from example-based modal synthesis paper (Lin)
Y = 6.2e10
density = 2600
thickness = 0.1

alpha = η = 1.8301e2
beta = γ = 1.4342e-7

k = Y*thickness

In [23]:
# Load file using my_util loader
V, faces, edges = load_obj(mesh_file)

In [24]:
# Extract other relevant information from the mesh including edge length and weight

edge_lengths = np.zeros((len(edges),))
for i, e in enumerate(edges):
    edge_lengths[i] = norm((V[e[0]] - V[e[1]]))

vertex_weights = np.zeros((len(V),))
for face in faces:
    # get area of the face
    a = V[face[0]]
    b = V[face[1]]
    c = V[face[2]]
    
    theta = angle_between(b-a, c-a)
    area = .5*norm(b-a)*norm(c-a)*math.sin(theta)
    
    for v in face:
        vertex_weights[v] += area

In [25]:
# Calculate the stiffness matrix
# (K_ij = k / l * v * v^T)

dim = len(V)*3
K = np.zeros((dim, dim))

for i, edge in enumerate(edges):
    # compute the direction unit vector
    unit = V[edge[1]] - V[edge[0]]
    unit /= norm(unit)
    unit.shape = (3, 1)
    
    submatrix = (k/edge_lengths[i])*unit.dot(unit.T)
    
    x = edge[0]*3
    y = edge[1]*3
    K[x:x+3, y:y+3] = submatrix
    K[y:y+3, x:x+3] = submatrix.T

# Calculate modes

In [ ]:
# Eigendecomposition of the stiffness matrix
D, G = eig(K)

In [16]:
# optional save the files
with open('cache/D.matrix', 'wb') as f:
    dump(D, f)

with open('cache/G.matrix', 'wb') as f:
    dump(G, f)

In [3]:
# optional load the files
with open('cache/D.matrix', 'rb') as f:
    D = load(f)

with open('cache/G.matrix', 'rb') as f:
    G = load(f)

In [8]:
# calculate the relevant mode information (frequencies, G_inv, damping coefficients)
eigs_pos = np.array([(-(γ*x+η) + np.sqrt((γ*x+η)*(γ*x+η) - (4*x) + 0J))/2 for x in D])
eigs_neg = np.array([(-(γ*x+η) - np.sqrt((γ*x+η)*(γ*x+η) - (4*x) + 0J))/2 for x in D])

freqs = np.array([abs(x)/(2*math.pi) for x in eigs_pos.imag])
damps = abs(eigs_pos.real)
max_damps = max(damps)
damps = np.array([d/max_damps for d in damps])
G_inv = np.linalg.inv(G.real)

bool_array = np.array([x>20 and x<22000 for x in freqs])

freqs = freqs[bool_array]
G_inv = G_inv[bool_array]
damps = damps[bool_array]

# Saving to File

### Mode File Format
`
number of vertices (n)
number of triangles (t)
number of modes (m)
vertex_0.x vertex_0.y vertex_0.z vertex_0.weight 
vertex_1.x vertex_1.y vertex_1.z vertex_1.weight
...  
vertex_(n-1).x vertex_(n-1).y vertex_(n-1).z vertex_(n-1).weight
triangle_0.index_0 triangle_0.index_1 triangle_0.index_2  
...  
triangle_(n-1).index_0 triangle_(n-1).index_1 triangle_(n-1).index_2
freq_0 (in Hz)
freq_1 
...
freq_(m-1)
damping_0
damping_1
...
damping(m-1)
G_(0,0) G_(1,0) ... G_((3*n)-1,0)
...     ...     ... ...
G_(0,m) G_(1,m) ... G_((3*n)-1,m)
END
`

#### Notes:
* damping normalized between 0 and 1
* only uses audible frequencies (relevant if scaling frequencies)
* G is actually G<sup>-1</sup>, and shaped to correspond to audible modes

In [11]:
def write_to_file(filename):
    with open(filename, 'w') as f:
        f.write(str(len(V))     + "\n")
        f.write(str(len(faces)) + "\n")
        f.write(str(len(freqs)) + "\n")
        for v, w in zip(V, vertex_weights):
            f.write(str(v[0]) + " " + str(v[1]) + " " + str(v[2]) + " " + str(w) + "\n")
        for face in faces:
            f.write(str(face[0]) + " " + str(face[1]) + " " + str(face[2]) + "\n")
        for hz in freqs:
            f.write(str(hz) + "\n")
        for d in damps:
            f.write(str(d) + "\n")
        for row in G_inv:
            for g in row:
                f.write(str(g) + " ")
            f.write("\n")
        f.write("END\n")

In [14]:
write_to_file("modes/alabama.modes")

NameError: name 'write_to_file' is not defined

# Write Reduced Modes

In [ ]:
rmodes = np.zeros(G_inv.T.shape)

In [ ]:
impulse = np.zeros((len(V)*3,))
impulse.shape = (len(V)*3, 1)
impulse[0] = 1
impulse[1] = 0
impulse[2] = 0

In [ ]:
# calculate final gain values and format relevant stuff as list (fgd_final)

gains = G_inv.dot(impulse)
pairs = [(f, abs(g[0])) for f, g in zip(freqs, gains)]
#m = max(lol,key = lambda x: x[1])[1]
#fgd_final = [(f, g[0], d) for f, g, d in zip(freqs, gains, damps)]

In [ ]:
rmodes = np.zeros((len(vertex_weights), len(freqs)))

max_gain = 0
for i in range(len(vertex_weights)):
    v = vertex_weights[i]
    impulse = np.zeros((len(V)*3,))
    impulse.shape = (len(V)*3, 1)
    impulse[0] = v
    impulse[1] = 0
    impulse[2] = 0
    
    gains = G_inv.dot(impulse)
    gains = abs(gains)
    
    m = max(gains)
    if m > max_gain:
        max_gain = m
    
    rmodes[i,:] = gains.T
    
rmodes = rmodes.T
rmodes /= m

In [ ]:
def write_reduced(filename):
    with open(filename, 'w') as f:
        f.write(str(len(V))     + "\n")
        f.write(str(len(faces)) + "\n")
        f.write(str(len(freqs)) + "\n")
        for v, w in zip(V, vertex_weights):
            f.write(str(v[0]) + " " + str(v[1]) + " " + str(v[2]) + " " + str(w) + "\n")
        for face in faces:
            f.write(str(face[0]) + " " + str(face[1]) + " " + str(face[2]) + "\n")
        for hz in freqs:
            f.write(str(hz) + "\n")
        for d in damps:
            f.write(str(d) + "\n")
        for row in rmodes:
            for g in row:
                f.write(str(g) + " ")
            f.write("\n")
        f.write("END\n")

In [ ]:
fname = "modes/railing.rmodes"
write_reduced(fname)

# Test Audio Output

In [91]:
# make sample impulse

impulse = np.zeros((len(V)*3,))
impulse.shape = (len(V)*3, 1)
impulse[len(V)*3//2] = 1
impulse[len(V)*3//2+1] = 1
impulse[len(V)*3//2+2] = 1
impulse[len(V)*3//2+12] = 1
impulse[len(V)*3//2+20] = 1
impulse[len(V)*3//2+21] = 1

In [118]:
# calculate final gain values and format relevant stuff as list (fgd_final)

gains = G_inv.dot(impulse)
lol = [(f, abs(g[0])) for f, g in zip(freqs, gains) if f > 20 and f < 20000]
m = max(lol,key = lambda x: x[1])[1]
fgd_final = [(f, g[0]/m, d) for f, g, d in zip(freqs, gains, damps) if f > 20 and f < 20000 and abs(g[0])/m > .1 and g[0] > 0]

In [120]:
SR = 44100
length = 1 # in seconds
samps = np.zeros((SR*length,))

d_scale = 0.01
f_scale = 1.0
for t in range(SR*length):
    samps[t] = sum([g*(math.sin(f_scale*(f*2*math.pi)*t/SR)*(math.e**(-(t/SR)*(d_scale/d)))) for f, g, d in fgd_final])
    
samps_max = max(samps)
samps /= samps_max;

filename = "sound_output/newer.wav"
scipy.io.wavfile.write(filename, SR, samps)